In [24]:
from dotenv import load_dotenv
from langchain import hub
#from langchain_chroma import Chroma
from langchain_community.document_loaders import  WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import JSONLoader
from langchain.prompts import ChatPromptTemplate

In [4]:
load_dotenv()

True

In [6]:
llm = ChatOpenAI(model="gpt-3.5-turbo")

In [9]:
import json
from pathlib import Path
from pprint import pprint


file_path='data.json'
data = json.loads(Path(file_path).read_text())

In [10]:
pprint(data)

[{'_run_ml_inference': True,
  'category': 'teams',
  'content': 'Effective: March 2020\n'
             'Purpose\n'
             '\n'
             'The purpose of this full-time work-from-home policy is to '
             'provide guidelines and support for employees to conduct their '
             'work remotely, ensuring the continuity and productivity of '
             'business operations during the COVID-19 pandemic and beyond.\n'
             'Scope\n'
             '\n'
             'This policy applies to all employees who are eligible for remote '
             'work as determined by their role and responsibilities. It is '
             'designed to allow employees to work from home full time while '
             'maintaining the same level of performance and collaboration as '
             'they would in the office.\n'
             'Eligibility\n'
             '\n'
             'Employees who can perform their work duties remotely and have '
             'received approval from 

In [17]:
loader = JSONLoader(
    file_path='data.json',
    jq_schema='.',
    text_content=False)

data2 = loader.load()

In [18]:
print(data)

[{'id': 1, 'content': "Effective: March 2020\nPurpose\n\nThe purpose of this full-time work-from-home policy is to provide guidelines and support for employees to conduct their work remotely, ensuring the continuity and productivity of business operations during the COVID-19 pandemic and beyond.\nScope\n\nThis policy applies to all employees who are eligible for remote work as determined by their role and responsibilities. It is designed to allow employees to work from home full time while maintaining the same level of performance and collaboration as they would in the office.\nEligibility\n\nEmployees who can perform their work duties remotely and have received approval from their direct supervisor and the HR department are eligible for this work-from-home arrangement.\nEquipment and Resources\n\nThe necessary equipment and resources will be provided to employees for remote work, including a company-issued laptop, software licenses, and access to secure communication tools. Employees 

In [20]:
import json
from langchain.schema import Document

# JSON dosyanızın yolu
file_path = "data.json"

# JSON verisini yükleme
with open(file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# JSON'daki her bir veriyi Document formatına çevirme
documents = [Document(page_content=json.dumps(entry, ensure_ascii=False), metadata={"source": file_path}) for entry in data]

# Sonuçları ekrana yazdır
for doc in documents:
    print(doc)


page_content='{"id": 1, "content": "Effective: March 2020\nPurpose\n\nThe purpose of this full-time work-from-home policy is to provide guidelines and support for employees to conduct their work remotely, ensuring the continuity and productivity of business operations during the COVID-19 pandemic and beyond.\nScope\n\nThis policy applies to all employees who are eligible for remote work as determined by their role and responsibilities. It is designed to allow employees to work from home full time while maintaining the same level of performance and collaboration as they would in the office.\nEligibility\n\nEmployees who can perform their work duties remotely and have received approval from their direct supervisor and the HR department are eligible for this work-from-home arrangement.\nEquipment and Resources\n\nThe necessary equipment and resources will be provided to employees for remote work, including a company-issued laptop, software licenses, and access to secure communication tool

In [22]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
splits= text_splitter.split_documents(data2)

In [23]:
from langchain_elasticsearch import ElasticsearchStore
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

vector_store= ElasticsearchStore.from_documents(
    splits,
    es_url="http://localhost:9200",
    index_name="my_basic_index",
    embedding=embeddings,
    es_user="elastic",
    es_password="changeme",
)


C:\Users\mt\AppData\Local\Temp\ipykernel_2880\3638780268.py:3: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


In [32]:

# 📌 Retriever Tanımla
retriever = vector_store.as_retriever()

# 📌 Prompt Şablonu
template = """
You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know.
Explain in 2-3 sentences.
Use five sentences minimum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:
"""
prompt = ChatPromptTemplate.from_template(template)


In [33]:
# 📌 RAG Zinciri
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [27]:
# 📌 Soru Sor ve Cevap Al
question = "Who is Nazlı?"
response = rag_chain.invoke(question)
print(response)

Nazlı is not mentioned in the provided context. Therefore, I do not have information about who Nazlı is.


In [34]:
# 📌 Soru Sor ve Cevap Al
question = "What is This career leveling matrix"
response = rag_chain.invoke(question)
print(response)

The career leveling matrix outlined in the document provides a structured framework for understanding the different roles and responsibilities of Software Engineers at various levels within an organization. It specifies the necessary skills and experience required for each level to support employee development, performance evaluations, and career progression paths. The matrix aims to guide Software Engineers in reaching their full potential and contributing significantly to the success of the organization by aligning their skills and experience with their career growth trajectory. It defines job levels such as Entry-Level, Intermediate-Level, Senior-Level, and Leadership-Level within distinct teams, each reflecting differing degrees of skills, experience, and responsibilities. Additionally, the matrix establishes compensation bands to ensure clear guidelines for salary ranges based on job levels and market benchmarks, promoting equitable compensation practices, employee growth, and per

In [35]:

# 📌 Soru Sor ve Cevap Al
question = "Which pets are included?"
response = rag_chain.invoke(question)
print(response)

The pets included in the office pet policy are dogs, cats, and other small, non-exotic animals, subject to approval by the HR department. Employees must follow the approval process, including submitting a written request with details about the pet, providing proof of vaccinations, and obtaining consent from coworkers sharing the workspace. The policy also outlines the behavior expectations, liability of pet owners for any damages, restricted areas where pets are not allowed, and guidelines for cleanliness and hygiene to maintain a safe and inclusive work environment. Employees are encouraged to direct any questions or concerns about the policy to their supervisor or the HR department for further clarification.
